In [2]:
import pandas as pd
from nebulagraph_lite import nebulagraph_let as ng_let
import os

# Data loading


In [2]:
#primekg = pd.read_csv("~/scratch-llm/data/PrimeKG_data/raw_data/kg.csv", low_memory=False)
nodes = pd.read_csv("~/scratch-llm/data/PrimeKG/datasets/data/kg/nodes.csv", low_memory=False)
edges = pd.read_csv("~/scratch-llm/data/PrimeKG/datasets/data/kg/edges.csv")

In [301]:
# load NebulaGraph JupyterNotebook extension
!udocker pull vesoft/nebula-metad:v3
!udocker create --name=nebula-metad vesoft/nebula-metad:v3
!udocker setup --execmode=F1 nebula-metad

!udocker pull vesoft/nebula-graphd:v3
!udocker create --name=nebula-graphd vesoft/nebula-graphd:v3
!udocker setup --execmode=F1 nebula-graphd

!udocker pull vesoft/nebula-storaged:v
!udocker create --name=nebula-storaged vesoft/nebula-storaged:v3
!udocker setup --execmode=F1 nebula-storaged

n = ng_let()
n.start() # This takes around 5 mins

15885.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Info: downloading layer sha256:20717c72714566f0a51c0f26f7237ae23515d730d9ffe7b8c34bb0f448d23beb
Info: downloading layer sha256:f73b0b86b2168fee37455b76bbb9a850d41ad48dda1544fa5a4c8fb933bfa006
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:3567ff3965abe5d66952e4a8be22af4c6e9a95d2ac02ada22ac6c0b0be37a2c1
Info: downloading layer sha256:5beb905c3081ead3dd9f9a60673fd722da67b135bc558bc71b92d69ec12ae999
Info: downloading layer sha256:2d473b07cdd5f0912cd6f1a703352c82b512407db6b05b43f2553732b55df3bc


15894.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


44d9cbe3-10dd-3c54-9a2b-0e5a7ca7299e


15957.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
15968.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

Error: pulling manifest:
Error: no files downloaded


16060.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


5c1eb597-aa8c-3243-9168-3cab0cc25f3c


16124.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Message: Activating storaged...
Result of `SHOW HOSTS`:
    errors:
        code: 0
    results:
        spaceName: 
        data:
            meta:
                None, None, None, None, None, None, None
            row:
                127.0.0.1, 9779, ONLINE, 141, PrimeKG:100, PrimeKG2:20, PrimeKG_manual:20, basketballplayer:1, PrimeKG:100, PrimeKG2:20, PrimeKG_manual:20, basketballplayer:1, 3.8.0
        columns:
            Host, Port, Status, Leader count, Leader distribution, Partition distribution, Version
        errors:
            code: 0
        latencyInUs: 1807
Info: loading basketballplayer dataset...

  _   _      _           _        ____                 _     
 | \ | | ___| |__  _   _| | __ _ / ___|_ __ __ _ _ __ | |__  
 |  \| |/ _ | '_ \| | | | |/ _` | |  _| '__/ _` | '_ \| '_ \ 
 | |\  |  __| |_) | |_| | | (_| | |_| | | | (_| | |_) | | | |
 |_| \_|\___|_.__/ \__,_|_|\__,_|\____|_|  \__,_| .__/|_| |_|
                                                |_|          
  

In [ ]:
%load_ext ngql

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql


In [3]:
%reload_ext ngql

In [4]:
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

[OK] Connection Pool Created


,Name
0,PrimeKG
1,PrimeKG2
2,PrimeKG_manual
3,basketballplayer


In [305]:
# Create a new Graph space (graph) for the PrimeKG
%ngql CREATE SPACE IF NOT EXISTS PrimeKG (vid_type = INT64);
%ngql USE PrimeKG;

""


# Create the graph from NebulaGraph directly

## Nodes

In [5]:
print(nodes['node_type'].unique()) #types of nodes

['gene/protein' 'drug' 'effect/phenotype' 'disease' 'biological_process'
 'molecular_function' 'cellular_component' 'exposure' 'pathway' 'anatomy']


In [ ]:
# save the data for each 'node_type' in a separate file
for node_type in nodes['node_type'].unique():
    sanitized_node_type = node_type.replace('/', '_')
    output_path = os.path.join('~/scratch-llm/data/PrimeKG_data/sub_data/', 'node_'+ sanitized_node_type + '.csv')
    nodes[nodes['node_type'] == node_type].to_csv(output_path, index=False)

In [ ]:
# Create Tags (node_type) for each node in the PrimeKG
%ngql CREATE TAG IF NOT EXISTS anatomy(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS gene_protein(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS drug(node_name string, node_source string, node_id string, description string, half_life string, indication string, mechanism_of_action string, protein_binding string, pharmacodynamics string, state string, atc_1 string, atc_2 string, atc_3 string, atc_4 string, category string, group string, pathway string, molecular_weight string, tpsa string, clogp string);
%ngql CREATE TAG IF NOT EXISTS disease(node_name string, node_source string, mondo_id int, mondo_name string, group_id_bert string, group_name_bert string, mondo_definition string, umls_description string, orphanet_definition string, orphanet_prevalence string, orphanet_epidemiology string, orphanet_clinical_description string, orphanet_management_and_treatment string, mayo_symptoms string, mayo_causes string, mayo_risk_factors string, mayo_complications string, mayo_prevention string, mayo_see_doc string);
%ngql CREATE TAG IF NOT EXISTS pathway(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS biological_process(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS effect_phenotype(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS molecular_function(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS cellular_component(node_name string, node_source string, node_id string);
%ngql CREATE TAG IF NOT EXISTS exposure(node_name string, node_source string, node_id string);

""


In [30]:
%ngql DESCRIBE TAG drug;

,Field,Type,Null,Default,Comment
0,node_name,string,YES,None,None
1,node_source,string,YES,None,None
2,node_id,string,YES,None,None
3,description,string,YES,None,None
4,half_life,string,YES,None,None
5,indication,string,YES,None,None
6,mechanism_of_action,string,YES,None,None
7,protein_binding,string,YES,None,None
8,pharmacodynamics,string,YES,None,None
9,state,string,YES,None,None


In [ ]:
# Load each node source data into the corresponding Tag (working fine)
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_gene_protein.csv --tag gene_protein --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_biological_process.csv --tag biological_process --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_effect_phenotype.csv --tag effect_phenotype --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_molecular_function.csv --tag molecular_function --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_cellular_component.csv --tag cellular_component --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_pathway.csv --tag pathway --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_exposure.csv --tag exposure --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_anatomy.csv --tag anatomy --header --space PrimeKG --vid 0 --props 2:node_id,3:node_name,4:node_source


### Extra information for nodes 'drug' and 'disease'

In [ ]:
# disease and drug feature files are available as .tab format so they need to be converted to .csv
# many formatting issues on both files that had to be correcteed manually before merging with the 'node_' dataset
disease_tab = "~/scratch-llm/data/PrimeKG_data/raw_data/disease_features.tab"
drug_tab = "~/scratch-llm/data/PrimeKG_data/raw_data/drug_features.tab"

df = pd.read_csv(disease_tab, delimiter='\t')
disease_csv = "~/scratch-llm/data/PrimeKG_data/sub_data/disease_features.csv"
df.to_csv(disease_csv, index=False)


df = pd.read_csv(drug_tab, delimiter='\t')
drug_csv = "~/scratch-llm/data/PrimeKG_data/sub_data/drug_features.csv"
df.to_csv(drug_csv, index=False) 

In [ ]:
# merge extra drug and disease information 
node_drug = pd.read_csv("~/scratch-llm/data/PrimeKG_data/sub_data/node_drug.csv")
features_drug = pd.read_csv("~/scratch-llm/data/PrimeKG_data/sub_data/drug_features.csv")
node_disease = pd.read_csv("~/scratch-llm/data/PrimeKG_data/sub_data/node_disease.csv")
features_disease = pd.read_csv("~/scratch-llm/data/PrimeKG_data/sub_data/disease_features.csv")

# Merge the dataframes on 'node_index' and save to csv
merged_df = pd.merge(node_drug, features_drug, on='node_index')
merged_df.to_csv("~/scratch-llm/data/PrimeKG_data/sub_data/node_merged_drug.csv", index=False)
merged_df = pd.merge(node_disease, features_disease, on='node_index')
merged_df.to_csv("~/scratch-llm/data/PrimeKG_data/sub_data/node_merged_disease.csv", index=False)


In [ ]:
#Load the extended drug and disease data into the corresponding Tags
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/merged_disease.csv --tag disease --header --space PrimeKG --vid 0 --props 3:node_name,4:node_source,5:mondo_id,6:mondo_name,7:group_id_bert,8:group_name_bert,9:mondo_definition,10:umls_description,11:orphanet_definition,12:orphanet_prevalence,13:orphanet_epidemiology,14:orphanet_clinical_description,15:orphanet_management_and_treatment,16:mayo_symptoms,17:mayo_causes,18:mayo_risk_factors,19:mayo_complications,20:mayo_prevention,21:mayo_see_doc
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/node_merged_drug.csv --tag drug --header --space PrimeKG --batch 100 --vid 0 --props 2:node_id,3:node_name,4:node_source,5:description,6:half_life,7:indication,8:mechanism_of_action,9:protein_binding,10:pharmacodynamics,11:state,12:atc_1,13:atc_2,14:atc_3,15:atc_4,16:category,17:group,18:pathway,19:molecular_weight,20:tpsa,21:clogp


## Edges

In [6]:
print(edges['relation'].unique()) # types of edges

['protein_protein' 'drug_protein' 'contraindication' 'indication'
 'off-label use' 'drug_drug' 'phenotype_protein' 'phenotype_phenotype'
 'disease_phenotype_negative' 'disease_phenotype_positive'
 'disease_protein' 'disease_disease' 'drug_effect' 'bioprocess_bioprocess'
 'molfunc_molfunc' 'cellcomp_cellcomp' 'molfunc_protein'
 'cellcomp_protein' 'bioprocess_protein' 'exposure_protein'
 'exposure_disease' 'exposure_exposure' 'exposure_bioprocess'
 'exposure_molfunc' 'exposure_cellcomp' 'pathway_pathway'
 'pathway_protein' 'anatomy_anatomy' 'anatomy_protein_present'
 'anatomy_protein_absent']


In [ ]:
# save the data for each 'relation' (edge type) in a separate file
for relation in edges['relation'].unique():
    sanitized_relation = relation.replace('-', '_')
    sanitized_relation = sanitized_relation.replace('\t', '_')
    output_path = os.path.join('~/scratch-llm/data/PrimeKG_data/sub_data/', 'edge_'+ sanitized_relation + '.csv')
    edges[edges['relation'] == relation].to_csv(output_path, index=False)

In [ ]:
%ngql CREATE EDGE IF NOT EXISTS protein_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS drug_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS contraindication(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS indication(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS off_label_use(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS drug_drug(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS phenotype_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS phenotype_phenotype(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS disease_phenotype_negative(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS disease_phenotype_positive(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS disease_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS disease_disease(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS drug_effect(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS bioprocess_bioprocess(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS molfunc_molfunc(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS cellcomp_cellcomp(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS molfunc_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS cellcomp_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS bioprocess_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_disease(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_exposure(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_bioprocess(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_molfunc(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS exposure_cellcomp(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS pathway_pathway(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS pathway_protein(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS anatomy_anatomy(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS anatomy_protein_present(display_relation string);
%ngql CREATE EDGE IF NOT EXISTS anatomy_protein_absent(display_relation string);

In [ ]:
# for each edge type, load the data accordingly, this takes like 20mins from new
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_protein_protein.csv --space PrimeKG --header --edge protein_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_drug_protein.csv --space PrimeKG --header --edge drug_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_contraindication.csv --space PrimeKG --header --edge contraindication --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_indication.csv --space PrimeKG --header --edge indication --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_off_label_use.csv --space PrimeKG --header --edge off_label_use --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_drug_drug.csv --space PrimeKG --header --edge drug_drug --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_phenotype_protein.csv --space PrimeKG --header --edge phenotype_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_phenotype_phenotype.csv --space PrimeKG --header --edge phenotype_phenotype --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_disease_phenotype_negative.csv --space PrimeKG --header --edge disease_phenotype_negative --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_disease_phenotype_positive.csv --space PrimeKG --header --edge disease_phenotype_positive --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_disease_protein.csv --space PrimeKG --header --edge disease_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_disease_disease.csv --space PrimeKG --header --edge disease_disease --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_drug_effect.csv --space PrimeKG --header --edge drug_effect --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_bioprocess_bioprocess.csv --space PrimeKG --header --edge bioprocess_bioprocess --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_molfunc_molfunc.csv --space PrimeKG --header --edge molfunc_molfunc --src 2 --dst 3 --props 1:display_relation


In [ ]:
# load the edge data in two batches
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_cellcomp_cellcomp.csv --space PrimeKG --header --edge cellcomp_cellcomp --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_molfunc_protein.csv --space PrimeKG --header --edge molfunc_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_cellcomp_protein.csv --space PrimeKG --header --edge cellcomp_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_bioprocess_protein.csv --space PrimeKG --header --edge bioprocess_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_protein.csv --space PrimeKG --header --edge exposure_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_disease.csv --space PrimeKG --header --edge exposure_disease --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_exposure.csv --space PrimeKG --header --edge exposure_exposure --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_bioprocess.csv --space PrimeKG --header --edge exposure_bioprocess --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_molfunc.csv --space PrimeKG --header --edge exposure_molfunc --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_exposure_cellcomp.csv --space PrimeKG --header --edge exposure_cellcomp --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_pathway_pathway.csv --space PrimeKG --header --edge pathway_pathway --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_pathway_protein.csv --space PrimeKG --header --edge pathway_protein --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_anatomy_anatomy.csv --space PrimeKG --header --edge anatomy_anatomy --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_anatomy_protein_present.csv --space PrimeKG --batch 500 --header --edge anatomy_protein_present --src 2 --dst 3 --props 1:display_relation
%ng_load --source ~/scratch-llm/data/PrimeKG_data/sub_data/edge_anatomy_protein_absent.csv --space PrimeKG --header --edge anatomy_protein_absent --src 2 --dst 3 --props 1:display_relation


## Checks

In [110]:
%ngql DESCRIBE TAG drug;

,Field,Type,Null,Default,Comment
0,node_name,string,YES,None,None
1,node_source,string,YES,None,None
2,node_id,string,YES,None,None
3,description,string,YES,None,None
4,half_life,string,YES,None,None
5,indication,string,YES,None,None
6,mechanism_of_action,string,YES,None,None
7,protein_binding,string,YES,None,None
8,pharmacodynamics,string,YES,None,None
9,state,string,YES,None,None


In [5]:
#small check-up, with extra drug and disease information for nodes
%ngql GET SUBGRAPH WITH PROP 1 STEPS FROM 14045 OUT drug_protein YIELD VERTICES AS nodes, EDGES AS relationships;

nodes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                relationships
0  [(14045 :drug{atc_1: __NULL__, atc_2: __NULL__, atc_3: __NULL__, atc_4: __NULL__, category: "Hydrocortisone phosphate is part of 11-Hydroxycorticosteroids ; 17-Hydroxycorticosteroids ; Adrenal Cortex Hormones ; Cytochrome P-450 CYP2C8 Inducers ; Cytochrome P-450 CYP2C8 Inducers (strength unknown) ; Cytochrome P-450 CYP3A Substrates ; Cytochrome P-450 CYP3A4 Substrates ; Cytochrome P-450 CYP3A5 Substrates ; Cytochrome P-450 CYP3A7 Substrates ; Cytochrome P-450 Enzyme Inducers ; Cytochrome P-450 Substrates ; Fused-Ring Compounds ; Hormones ; Hormones, Hormone Substitutes, and Hormone Antagonists ; Hydrocortisone and derivatives ; Hydroxycorticosteroids ; Pregnanes ; Pregnenediones ; Pregnenes ; Steroids.", description: __NULL__, group: "Hydrocortisone phosphate is approved and vet_approved.", half_life: "The half-life is 6-8 hours", indication: "For the relief of the inflammatory and pruritic manifestations of corticosteroid-responsive dermatoses. Also used to treat endocrine (hormonal) disorders (adrenal insufficiency, Addisons disease). It is also used to treat many immune and allergic disorders, such as arthritis, lupus, severe psoriasis, severe asthma, ulcerative colitis, and Crohn's disease.", mechanism_of_action: "Hydrocortisone binds to the cytosolic glucocorticoid receptor. After binding the receptor the newly formed receptor-ligand complex translocates itself into the cell nucleus, where it binds to many glucocorticoid response elements (GRE) in the promoter region of the target genes. The DNA bound receptor then interacts with basic transcription factors, causing the increase in expression of specific target genes. The anti-inflammatory actions of corticosteroids are thought to involve lipocortins, phospholipase A2 inhibitory proteins which, through inhibition arachidonic acid, control the biosynthesis of prostaglandins and leukotrienes. Specifically glucocorticoids induce lipocortin-1 (annexin-1) synthesis, which then binds to cell membranes preventing the phospholipase A2 from coming into contact with its substrate arachidonic acid. This leads to diminished eicosanoid production. The cyclooxygenase (both COX-1 and COX-2) expression is also suppressed, potentiating the effect. In other words, the two main products in inflammation Prostaglandins and Leukotrienes are inhibited by the action of Glucocorticoids. Glucocorticoids also stimulate the lipocortin-1 escaping to the extracellular space, where it binds to the leukocyte membrane receptors and inhibits various inflammatory events: epithelial adhesion, emigration, chemotaxis, phagocytosis, respiratory burst and the release of various inflammatory mediators (lysosomal enzymes, cytokines, tissue plasminogen activator, chemokines etc.) from neutrophils, macrophages and mastocytes. Additionally the immune system is suppressed by corticosteroids due to a decrease in the function of the lymphatic system, a reduction in immunoglobulin and complement concentrations, the precipitation of lymphocytopenia, and interference with antigen-antibody binding.", molecular_weight: "The

# Load directly to NebulaPropertyGraphStore

In [6]:
from llama_index.core.schema import NodeRelationship, TextNode, IndexNode, RelatedNodeInfo
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore
from llama_index.core import (
    StorageContext,
)

In [306]:
%ngql CREATE SPACE IF NOT EXISTS PrimeKG_manual(vid_type=FIXED_STRING(30), partition_num=20, replica_factor=1);
%ngql use PrimeKG_manual;

""


In [7]:
# PropertyGraphIndex requires NebulaPropertyGraphStore
graph_store = NebulaPropertyGraphStore(
    space= "PrimeKG_manual", 
    username = "root",
    password = "nebula",
    url = "nebula://localhost:9669",
    props_schema= "`node_id` STRING, `node_name` STRING, `node_source` STRING, `mondo_id` STRING, `mondo_name` STRING, `group_id_bert` STRING, `group_name_bert` STRING, `mondo_definition` STRING, `umls_description` STRING, `orphanet_definition` STRING, `orphanet_prevalence` STRING, `orphanet_epidemiology` STRING, `orphanet_clinical_description` STRING, `orphanet_management_and_treatment` STRING, `mayo_symptoms` STRING, `mayo_causes` STRING, `mayo_risk_factors` STRING, `mayo_complications` STRING, `mayo_prevention` STRING, `mayo_see_doc` STRING, `display_relation` STRING, `description` STRING, `half_life` STRING, `indication` STRING, `mechanism_of_action` STRING, `protein_binding` STRING, `pharmacodynamics` STRING, `state` STRING, `atc_1` STRING, `atc_2` STRING, `atc_3` STRING, `atc_4` STRING, `category` STRING, `group` STRING, `pathway` STRING, `molecular_weight` STRING, `tpsa` STRING, `clogp` STRING"
)

storage_context = StorageContext.from_defaults(property_graph_store=graph_store)

define files to use:

In [10]:
data_dir = '~/scratch-llm/data/PrimeKG_data/sub_data/'

# List of node types and their corresponding CSV files
node_files = {
    'gene_protein': 'node_gene_protein.csv',
    'drug': 'node_merged_drug.csv', # manually corrected and generated from "extra information section"
    'effect_phenotype': 'node_effect_phenotype.csv',
    'disease': 'node_merged_disease.csv', # manually corrected and generated from "extra information section"
    'biological_process': 'node_biological_process.csv',
    'molecular_function': 'node_molecular_function.csv',
    'cellular_component': 'node_cellular_component.csv',
    'exposure': 'node_exposure.csv',
    'pathway': 'node_pathway.csv',
    'anatomy': 'node_anatomy.csv'
}

edge_files ={
    'protein_protein': 'edge_protein_protein.csv',
    'drug_protein': 'edge_drug_protein.csv',
    'contraindication': 'edge_contraindication.csv',
    'indication': 'edge_indication.csv',
    'off_label_use': 'edge_off_label_use.csv',
    'drug_drug': 'edge_drug_drug.csv',
    'phenotype_protein': 'edge_phenotype_protein.csv',
    'phenotype_phenotype': 'edge_phenotype_phenotype.csv',
    'disease_phenotype_negative': 'edge_disease_phenotype_negative.csv',
    'disease_phenotype_positive': 'edge_disease_phenotype_positive.csv',
    'disease_protein': 'edge_disease_protein.csv',
    'disease_disease': 'edge_disease_disease.csv',
    'drug_effect': 'edge_drug_effect.csv',
    'bioprocess_bioprocess': 'edge_bioprocess_bioprocess.csv',
    'molfunc_molfunc': 'edge_molfunc_molfunc.csv',
    'cellcomp_cellcomp': 'edge_cellcomp_cellcomp.csv',
    'molfunc_protein': 'edge_molfunc_protein.csv',
    'cellcomp_protein': 'edge_cellcomp_protein.csv',
    'bioprocess_protein': 'edge_bioprocess_protein.csv',
    'exposure_protein': 'edge_exposure_protein.csv',
    'exposure_disease': 'edge_exposure_disease.csv',
    'exposure_exposure': 'edge_exposure_exposure.csv',
    'exposure_bioprocess': 'edge_exposure_bioprocess.csv',
    'exposure_molfunc': 'edge_exposure_molfunc.csv',
    'exposure_cellcomp': 'edge_exposure_cellcomp.csv',
    'pathway_pathway': 'edge_pathway_pathway.csv',
    'pathway_protein': 'edge_pathway_protein.csv',
    'anatomy_anatomy': 'edge_anatomy_anatomy.csv',
    'anatomy_protein_present': 'edge_anatomy_protein_present.csv',
    'anatomy_protein_absent': 'edge_anatomy_protein_absent.csv'
}

## Nodes

In [11]:
# create TextNodes
def create_text_nodes(file_path):
    df = pd.read_csv(file_path)
    text_nodes = []
    for _, row in df.iterrows():
        id = row['node_index']
        # if row length is greater than 4, then it is drug/disease node
        if len(row) > 4:
            metadata = row[['node_index','node_type','node_name','node_source']].to_dict()
            text = str(row.drop(['node_id','node_index','node_type','node_name','node_source']).to_dict())
            text_node = TextNode(id_= str(id), metadata=metadata, text=text)
            text_nodes.append(text_node)
        else:
            metadata = row.drop(['node_index']).to_dict()
            text_node = TextNode(id_= str(id), metadata=metadata)
            text_nodes.append(text_node)
    return text_nodes


In [12]:
# generate all nodes from the node files
all_nodes = []
for key, value in node_files.items():
    nodes = create_text_nodes(data_dir+value)
    all_nodes.extend(nodes)


In [14]:
all_nodes[34932]

TextNode(id_='21273', embedding=None, metadata={'node_index': 21273, 'node_type': 'drug', 'node_name': 'Magnesium Aluminum Silicate', 'node_source': 'DrugBank'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="{'description': 'Magnesium aluminum silicate is a naturally occurring mineral obtained from silicate ores of the montmorillonite group. It is refined to a powder for use in cosmetic and pharmaceutical applications as an absorbent, anticaking agent, opacifying agent, viscosity-increasing agent, suspending agent, tablet and capsule disintegrant, and', 'half_life': nan, 'indication': 'Magnesium aluminum silicate is used as an over the counter antacid for the self-treatment of heartburn, sour stomach, or acid indigestion.', 'mechanism_of_action': 'Magnesium aluminum silicate reacts with acid in the stomach to increase the pH.', 'protein_binding': nan, 'pharmacodynamics': 'Magnesium aluminum silicate neutralizes excess acid in the stomach to red

## Edges

In [17]:
def create_all_edges(edge_files, data_dir, all_nodes):
    node_dict = {node.id_: node for node in all_nodes}
    
    for edge_type, file_name in edge_files.items():
        file_path = os.path.join(data_dir, file_name)
        df = pd.read_csv(file_path)
        
        for _, row in df.iterrows():
            src_id = str(row['x_index'])
            dst_id = str(row['y_index'])
            metadata = row.drop(['x_index', 'y_index']).to_dict()
            
            if src_id in node_dict and dst_id in node_dict:
                src_node = node_dict[src_id]
                dst_node = node_dict[dst_id]
                
                if NodeRelationship.NEXT not in src_node.relationships:
                    src_node.relationships[NodeRelationship.NEXT] = []
                if NodeRelationship.PREVIOUS not in dst_node.relationships:
                    dst_node.relationships[NodeRelationship.PREVIOUS] = []
                
                src_node.relationships[NodeRelationship.NEXT].append(RelatedNodeInfo(node_id=dst_id, metadata=metadata))
                dst_node.relationships[NodeRelationship.PREVIOUS].append(RelatedNodeInfo(node_id=src_id, metadata=metadata))


In [ ]:
# Call the function to create all edges
create_all_edges(edge_files, data_dir, all_nodes)

check ups that edges are correctly generated:

In [ ]:
%ngql use PrimeKG;
%ngql GET SUBGRAPH WITH PROP 1 STEPS FROM 14000 OUT protein_protein YIELD VERTICES AS nodes, EDGES AS relationships;

,nodes,relationships
0,[(14000 )],"[(14000)-[:protein_protein@0{display_relation: ""ppi""}]->(6769), (14000)-[:protein_protein@0{display_relation: ""ppi""}]->(1411), (14000)-[:protein_protein@0{display_relation: ""ppi""}]->(7073), (14000)-[:protein_protein@0{display_relation: ""ppi""}]->(35314)]"
1,"[(1411 ), (6769 ), (35314 ), (7073 )]","[(1411)-[:protein_protein@0{display_relation: ""ppi""}]->(14000), (6769)-[:protein_protein@0{display_relation: ""ppi""}]->(14000), (35314)-[:protein_protein@0{display_relation: ""ppi""}]->(14000), (7073)-[:protein_protein@0{display_relation: ""ppi""}]->(14000)]"


In [12]:
all_nodes[14000].relationships

{<NodeRelationship.PREVIOUS: '2'>: [RelatedNodeInfo(node_id='7073', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInfo(node_id='1411', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInfo(node_id='6769', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInfo(node_id='35314', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None)],
 <NodeRelationship.NEXT: '3'>: [RelatedNodeInfo(node_id='35314', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInfo(node_id='7073', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInfo(node_id='1411', node_type=None, metadata={'relation': 'protein_protein', 'display_relation': 'ppi'}, hash=None),
  RelatedNodeInf

## Generate index

convert TextNode into IndexNode to insert the nodes into the index:

In [ ]:
from llama_index.core.vector_stores.simple import SimpleVectorStore
from llama_index.core import PropertyGraphIndex, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch


vec_store = SimpleVectorStore()

In [14]:
# Create IndexNode objects from TextNode objects
index_nodes = [IndexNode.from_text_node(node, index_id="index_1") for node in all_nodes]


In [15]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.8, "do_sample": True},
    # system_prompt=system_prompt,
    # query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-3.2-3B",
    model_name="meta-llama/Llama-3.2-3B",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 1024
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3") 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

index = PropertyGraphIndex(
    nodes=index_nodes,
    vector_store=vec_store,
    property_graph_store=graph_store,
    show_progress=True,
    storage_context=storage_context
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

index = PropertyGraphIndex.from_existing(
    llm=llm,
    vector_store=vec_store,
    property_graph_store=graph_store,
    show_progress=True,
    storage_context=storage_context
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

await index.insert_nodes(index_nodes)

save index to persist storage

In [ ]:
index.storage_context.persist(persist_dir="~/scratch-llm/storage/PrimeKG_index/")
